#This notebook uses Gemini 1.5 Flash and covers function calling using Gemini api

##Setup

In [ ]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 6.8 MB/s eta 0:00:00


In [40]:
import google.generativeai as genai

###To run this cell, your API key needs to be stored in the Colab Secret named as GOOGLE_API_KEY
###To get the key, go to ai.google.dev

In [41]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key= GOOGLE_API_KEY)

##Single Function

####I defined functions relating to a hypothetical restaurant called Italiano

In [42]:
def get_full_menu(service: str):
    """List all items on the menu of Italiano for the given service.

    Args:
        name: The type of service, lunch or dinner.
    """
    return ["Chicken Caesar Salad", "Margherita Pizza", "Spaghetti and Meatballs", "Eggplant Parmesan"]


def find_vegetarian_items(items: list[str]):
    """List all dishes in items that are vegetarian.

    Args:
        items: A list of dinner dishes.
    """
    return ["Margherita Pizza", "Eggplant Parmesan"]

def enter_restaurant():
    """You enter Gemini's Trattoria, moving the creaky door."""
    print("The door swings open, making a loud noise.")
    return True

functions = {"get_full_menu": get_full_menu,
             "find_vegetarian_items": find_vegetarian_items,
             "enter_restaurant": enter_restaurant}

##Model

In [43]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=functions.values() #giving the model access to the functions
)

chat = model.start_chat(enable_automatic_function_calling=True)


###Sending a prompt that requires user-defined functions

In [44]:
response = chat.send_message(
    "What items are on Italiano's dinner menu?"
)
print(response.text)

The dinner menu at Italiano has the following items: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, Eggplant Parmesan. 



###The chat history, the model initially sends back a function call, to which we automatically respond, which then leads to the final model output.

In [45]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items are on Italiano's dinner menu?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': 'The dinner menu at Italiano has the following items: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Me

In general, the processes goes as follows:
1. The user submits a query to the model.
2. The model responds with a function call.
3. The user runs the function and returns the result of the function.
4. Now, the model will either go back to Step 2 or output a final response, as seen above.

##Multiple function calls
###This process can happen either at the same time or one after the other

###Example 1:

In [46]:
response = chat.send_message(
    #Asking about the DINNER menu item and VEGETERIAN
    "What are the vegetarian items on Italiano's dinner menu?"
)
print(response.text)

The vegetarian items on Italiano's dinner menu are Margherita Pizza and Eggplant Parmesan. 



###The chat history of multiple function calls

In [47]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items are on Italiano's dinner menu?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': 'The dinner menu at Italiano has the following items: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Me

###*As the above history shows history of multiple prompts, we can create a new instance of the chat variable  

In [48]:
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message(
    #Asking about the DINNER menu item and VEGETERIAN
    "What are the vegetarian items on Italiano's dinner menu?"
)
print(response.text)

The vegetarian items on Italiano's dinner menu are Margherita Pizza and Eggplant Parmesan. 



In [49]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What are the vegetarian items on Italiano's dinner menu?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'find_vegetarian_items', 'args': {'items': ['Chicken Caesar Salad', 'Margheri

###Example 2:

In [50]:
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "Your are standing outside of Italiano restaurant. Enter the restaurant and read out the items on the menu."
)
print(response.text)

The door swings open, making a loud noise.
Welcome to Italiano!  Today's lunch menu is: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan. 



In [51]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': 'Your are standing outside of Italiano restaurant. Enter the restaurant and read out the items on the menu.'}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}}}, {'function_call': {'name': 'get_full_menu', 'args': {'service': 'lunch'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'enter_restaurant', 'response': {'result': True}}}, {'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------

##Function calling configuration
###Allows us to control how the Gemini API acts when tools (function) have been specified.
###3 types of config will be explored:
* none
* auto
* any
* manually defining accessible functions

##Function calling config Setup

In [52]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

###none: the model knows there are functions but cannot access them

In [53]:
tool_config = tool_config_from_mode("none")
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Italiano's dinner menu are vegetarian?", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items on Italiano's dinner menu are vegetarian?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': "Please provide me with the menu for Italiano's restaurant. I need the menu to identify the vegetarian options. \n"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


###auto: the model can decide whether to reply using text or specific functions

In [54]:
tool_config = tool_config_from_mode("auto")
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Italiano's dinner menu are pasta dishes?", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items on Italiano's dinner menu are pasta dishes?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': "The pasta dishes on Italiano's dinner menu are Spaghetti and Meatballs and Eggplant Parmesan. \n

###any: the model is forced to make function calls.
####I used Gemini 1.5 pro to achieve the intended behavior

In [55]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro", tools=functions.values()
)

tool_config = tool_config_from_mode("any")
chat = model.start_chat()

response = chat.send_message(
    "Enter Italiano.", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': 'Enter Italiano.'}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


###Manually setting allowed_function_names: the model will only choose from those functions. The model is forced to use the specified function even when its not best to use

In [56]:
tool_config = tool_config_from_mode("any", ["get_full_menu"])
chat = model.start_chat()

response = chat.send_message(
    "Enter Italiano.", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': 'Enter Italiano.'}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'lunch'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
